In [1]:
""" First tests with WRN - 10-2. """
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
import wrn_high as wrn
import keras.callbacks as callbacks

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
#data augmentation
datagen = ImageDataGenerator(rotation_range=5,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             horizontal_flip=True,
                             fill_mode="reflect")
datagen.fit(x_train)

In [3]:
def schedule(x):
    if x < 45:
        return 0.125
    elif x < 85:
        return 0.125*0.2
    elif x < 105:
        return 0.125*0.2*0.2
    elif x < 125:
        return 0.125*0.2*0.2*0.2
    else:
        return 0.125*0.2*0.2*0.2*0.5

In [4]:
def run(act):# SWISH - 125
    init_shape = (3, 32, 32) if K.image_dim_ordering() == 'th' else (32, 32, 3)
    # For WRN-16-8 put N = 2, k = 8
    # For WRN-28-10 put N = 4, k = 10
    # For WRN-40-4 put N = 6, k = 4
    model = wrn.build_model(init_shape, num_classes, 10, 2)

    print("Model Created")
    batch_size  = 128
    epochs = 125

    opt = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.0, nesterov=False)
    lr_1 = keras.callbacks.LearningRateScheduler(schedule)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    print("Finished compiling")

    ####################
    # Network training #
    ####################

    print("Gonna fit the model")
    his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
#     print(his.history)
    return his.history

In [7]:
run("act")

Model Created
Finished compiling
Gonna fit the model
Epoch 1/125
390/390 [==============================] - 35s 91ms/step - loss: 1.4636 - acc: 0.4623 - val_loss: 1.3533 - val_acc: 0.5576
Epoch 2/125
390/390 [==============================] - 35s 90ms/step - loss: 1.0228 - acc: 0.6376 - val_loss: 1.0025 - val_acc: 0.6565
Epoch 3/125
390/390 [==============================] - 35s 90ms/step - loss: 0.8504 - acc: 0.7041 - val_loss: 0.8529 - val_acc: 0.7124
Epoch 4/125
390/390 [==============================] - 35s 90ms/step - loss: 0.7550 - acc: 0.7360 - val_loss: 1.2492 - val_acc: 0.6346
Epoch 5/125
390/390 [==============================] - 35s 90ms/step - loss: 0.6852 - acc: 0.7618 - val_loss: 0.6488 - val_acc: 0.7781
Epoch 6/125
390/390 [==============================] - 35s 90ms/step - loss: 0.6305 - acc: 0.7806 - val_loss: 0.8291 - val_acc: 0.7328
Epoch 7/125
390/390 [==============================] - 35s 90ms/step - loss: 0.5941 - acc: 0.7931 - val_loss: 0.6204 - val_acc: 0.7932
Ep

Epoch 61/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1676 - acc: 0.9411 - val_loss: 0.3424 - val_acc: 0.8949
Epoch 62/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1633 - acc: 0.9428 - val_loss: 0.3396 - val_acc: 0.8932
Epoch 63/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1606 - acc: 0.9443 - val_loss: 0.3320 - val_acc: 0.9006
Epoch 64/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1605 - acc: 0.9436 - val_loss: 0.3330 - val_acc: 0.9003
Epoch 65/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1602 - acc: 0.9439 - val_loss: 0.3335 - val_acc: 0.9003
Epoch 66/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1644 - acc: 0.9426 - val_loss: 0.3392 - val_acc: 0.8987
Epoch 67/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1578 - acc: 0.9447 - val_loss: 0.3306 - val_acc: 0.8999
Epoch 68/125
390/390 [==========================

390/390 [==============================] - 35s 90ms/step - loss: 0.1195 - acc: 0.9581 - val_loss: 0.3327 - val_acc: 0.9056
Epoch 122/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1193 - acc: 0.9580 - val_loss: 0.3332 - val_acc: 0.9050
Epoch 123/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1192 - acc: 0.9578 - val_loss: 0.3338 - val_acc: 0.9059
Epoch 124/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1206 - acc: 0.9593 - val_loss: 0.3325 - val_acc: 0.9055
Epoch 125/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1200 - acc: 0.9585 - val_loss: 0.3344 - val_acc: 0.9054


{'acc': [0.46216313761321637,
  0.63755213342341699,
  0.70414260504985415,
  0.73594401664446285,
  0.761810234161306,
  0.7805983317101044,
  0.79299005453962146,
  0.80596326592261491,
  0.81534728905999354,
  0.82023981392364453,
  0.82789942248341053,
  0.836822264960151,
  0.84069217199846302,
  0.84540423487943839,
  0.8530838947323679,
  0.85669313446237061,
  0.85741498239307323,
  0.86166586457516547,
  0.86383140838639572,
  0.86762111000346187,
  0.87088947704215447,
  0.87289461020211745,
  0.87690487648379856,
  0.87738610841847786,
  0.88408325310888525,
  0.8821984280138625,
  0.88652951555983317,
  0.88701074747538999,
  0.88729146611510767,
  0.8897577799548313,
  0.894951074732241,
  0.89326676287481255,
  0.89467035609252332,
  0.89783846651241872,
  0.89739733720231141,
  0.89968318892550236,
  0.89878087903124648,
  0.90096647419300757,
  0.90499679178697467,
  0.90565848568520713,
  0.90461581649021494,
  0.90674125758126101,
  0.9074230028491469,
  0.90988931661

In [8]:
run("act")

Model Created
Finished compiling
Gonna fit the model
Epoch 1/125
390/390 [==============================] - 35s 91ms/step - loss: 1.4812 - acc: 0.4551 - val_loss: 1.6290 - val_acc: 0.4853
Epoch 2/125
390/390 [==============================] - 35s 90ms/step - loss: 1.0406 - acc: 0.6292 - val_loss: 1.0975 - val_acc: 0.6407
Epoch 3/125
390/390 [==============================] - 35s 90ms/step - loss: 0.8597 - acc: 0.6945 - val_loss: 0.8614 - val_acc: 0.7006
Epoch 4/125
390/390 [==============================] - 35s 90ms/step - loss: 0.7603 - acc: 0.7325 - val_loss: 1.1223 - val_acc: 0.6097
Epoch 5/125
390/390 [==============================] - 35s 90ms/step - loss: 0.6915 - acc: 0.7601 - val_loss: 1.2396 - val_acc: 0.6442
Epoch 6/125
390/390 [==============================] - 35s 90ms/step - loss: 0.6447 - acc: 0.7758 - val_loss: 0.8798 - val_acc: 0.7202
Epoch 7/125
390/390 [==============================] - 35s 90ms/step - loss: 0.5972 - acc: 0.7934 - val_loss: 0.8105 - val_acc: 0.7370
Ep

Epoch 61/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1675 - acc: 0.9411 - val_loss: 0.3609 - val_acc: 0.8927
Epoch 62/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1629 - acc: 0.9416 - val_loss: 0.3461 - val_acc: 0.8896
Epoch 63/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1661 - acc: 0.9417 - val_loss: 0.3618 - val_acc: 0.8902
Epoch 64/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1638 - acc: 0.9429 - val_loss: 0.3723 - val_acc: 0.8899
Epoch 65/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1620 - acc: 0.9430 - val_loss: 0.3712 - val_acc: 0.8901
Epoch 66/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1634 - acc: 0.9423 - val_loss: 0.3391 - val_acc: 0.8973
Epoch 67/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1619 - acc: 0.9420 - val_loss: 0.3369 - val_acc: 0.8946
Epoch 68/125
390/390 [==========================

390/390 [==============================] - 35s 90ms/step - loss: 0.1205 - acc: 0.9564 - val_loss: 0.3502 - val_acc: 0.8990
Epoch 122/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1227 - acc: 0.9577 - val_loss: 0.3520 - val_acc: 0.8989
Epoch 123/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1201 - acc: 0.9576 - val_loss: 0.3520 - val_acc: 0.8988
Epoch 124/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1177 - acc: 0.9587 - val_loss: 0.3515 - val_acc: 0.8993
Epoch 125/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1235 - acc: 0.9568 - val_loss: 0.3517 - val_acc: 0.8999


{'acc': [0.45516522298036649,
  0.62925088225858195,
  0.69449791462328869,
  0.73243503368623675,
  0.76016602504318409,
  0.77584134615384615,
  0.79337267174733017,
  0.80068976580044915,
  0.81526708369611511,
  0.82174366376015251,
  0.82296679497606529,
  0.8328721526726951,
  0.83948909207571387,
  0.84311838307975762,
  0.84805101056798049,
  0.8515800449149824,
  0.85663298042990055,
  0.85753205128205123,
  0.86237556199730403,
  0.86385145975604893,
  0.8649943855889608,
  0.87113009303817779,
  0.87223291624651755,
  0.87381697142778159,
  0.87850898301559344,
  0.88177735003516355,
  0.88223853065755686,
  0.8864092076096276,
  0.88472489573307667,
  0.88890224358974357,
  0.89141779060385207,
  0.89274542831556136,
  0.89080044913070111,
  0.89471045875534017,
  0.89823949312146445,
  0.90012431825473216,
  0.90038498558216384,
  0.90136750076380834,
  0.90154796276560645,
  0.90218349358974359,
  0.9033397560174754,
  0.90537776712197926,
  0.90593920438229214,
  0.90527

In [9]:
# Using He initialization
run("act")

Model Created
Finished compiling
Gonna fit the model
Epoch 1/125
390/390 [==============================] - 35s 91ms/step - loss: 1.4940 - acc: 0.4544 - val_loss: 1.4668 - val_acc: 0.5072
Epoch 2/125
390/390 [==============================] - 35s 90ms/step - loss: 1.0675 - acc: 0.6198 - val_loss: 1.4434 - val_acc: 0.5801
Epoch 3/125
390/390 [==============================] - 35s 90ms/step - loss: 0.8773 - acc: 0.6913 - val_loss: 1.0260 - val_acc: 0.6340
Epoch 4/125
390/390 [==============================] - 35s 90ms/step - loss: 0.7745 - acc: 0.7304 - val_loss: 0.7134 - val_acc: 0.7496
Epoch 5/125
390/390 [==============================] - 35s 90ms/step - loss: 0.7033 - acc: 0.7547 - val_loss: 1.0043 - val_acc: 0.6880
Epoch 6/125
390/390 [==============================] - 35s 90ms/step - loss: 0.6596 - acc: 0.7692 - val_loss: 0.7626 - val_acc: 0.7510
Epoch 7/125
390/390 [==============================] - 35s 90ms/step - loss: 0.6146 - acc: 0.7872 - val_loss: 0.9213 - val_acc: 0.7130
Ep

Epoch 61/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1706 - acc: 0.9395 - val_loss: 0.3379 - val_acc: 0.8975
Epoch 62/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1709 - acc: 0.9402 - val_loss: 0.3501 - val_acc: 0.8959
Epoch 63/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1704 - acc: 0.9392 - val_loss: 0.3442 - val_acc: 0.8973
Epoch 64/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1682 - acc: 0.9407 - val_loss: 0.3307 - val_acc: 0.9019
Epoch 65/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1658 - acc: 0.9410 - val_loss: 0.3424 - val_acc: 0.8994
Epoch 66/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1659 - acc: 0.9415 - val_loss: 0.3350 - val_acc: 0.8998
Epoch 67/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1647 - acc: 0.9423 - val_loss: 0.3375 - val_acc: 0.8990
Epoch 68/125
390/390 [==========================

390/390 [==============================] - 35s 90ms/step - loss: 0.1221 - acc: 0.9572 - val_loss: 0.3386 - val_acc: 0.9045
Epoch 122/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1239 - acc: 0.9561 - val_loss: 0.3379 - val_acc: 0.9046
Epoch 123/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1234 - acc: 0.9573 - val_loss: 0.3399 - val_acc: 0.9049
Epoch 124/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1234 - acc: 0.9567 - val_loss: 0.3410 - val_acc: 0.9030
Epoch 125/125
390/390 [==============================] - 35s 90ms/step - loss: 0.1238 - acc: 0.9559 - val_loss: 0.3389 - val_acc: 0.9035


{'acc': [0.45432306705165221,
  0.61974655117099775,
  0.69128970163618864,
  0.73044871794871791,
  0.75463631980757573,
  0.76924927813294686,
  0.78715511709977537,
  0.79894529998704034,
  0.80797275641025645,
  0.81412572256249349,
  0.82274623033057281,
  0.83010506899570247,
  0.83565928779583076,
  0.83868703878113271,
  0.84255694578119988,
  0.84949470644850822,
  0.85334456211716692,
  0.85603144052589331,
  0.85982114208559213,
  0.86345043314700332,
  0.86579643886441948,
  0.86874398460057745,
  0.870147577760921,
  0.87459935897435892,
  0.87785003211303791,
  0.87862929098492137,
  0.88141642603156733,
  0.88326114855938553,
  0.88346166187346953,
  0.88767244148835722,
  0.886810234161306,
  0.89037937115200216,
  0.89064003847943385,
  0.89178685897435894,
  0.89312379570365641,
  0.89545235799832879,
  0.89886108435688017,
  0.89854026307346813,
  0.89972329160744158,
  0.90313201800423781,
  0.90036493426987785,
  0.90439525184472247,
  0.90423484119345521,
  0.9055

In [5]:
run("act")

Model Created
Finished compiling
Gonna fit the model
Epoch 1/125
390/390 [==============================] - 38s 98ms/step - loss: 1.4768 - acc: 0.4611 - val_loss: 1.2710 - val_acc: 0.5389
Epoch 2/125
390/390 [==============================] - 37s 95ms/step - loss: 1.0489 - acc: 0.6280 - val_loss: 1.0244 - val_acc: 0.6582
Epoch 3/125
390/390 [==============================] - 37s 95ms/step - loss: 0.8856 - acc: 0.6896 - val_loss: 1.2191 - val_acc: 0.6441
Epoch 4/125
390/390 [==============================] - 37s 95ms/step - loss: 0.7900 - acc: 0.7241 - val_loss: 0.7798 - val_acc: 0.7328
Epoch 5/125
390/390 [==============================] - 37s 94ms/step - loss: 0.7344 - acc: 0.7438 - val_loss: 0.8886 - val_acc: 0.7097
Epoch 6/125
390/390 [==============================] - 37s 95ms/step - loss: 0.6785 - acc: 0.7619 - val_loss: 0.8854 - val_acc: 0.7118
Epoch 7/125
390/390 [==============================] - 37s 95ms/step - loss: 0.6376 - acc: 0.7794 - val_loss: 0.7887 - val_acc: 0.7432
Ep

Epoch 61/125
390/390 [==============================] - 37s 94ms/step - loss: 0.2213 - acc: 0.9219 - val_loss: 0.4686 - val_acc: 0.8735
Epoch 62/125
390/390 [==============================] - 37s 94ms/step - loss: 0.2217 - acc: 0.9226 - val_loss: 0.4080 - val_acc: 0.8831
Epoch 63/125
390/390 [==============================] - 37s 94ms/step - loss: 0.2178 - acc: 0.9234 - val_loss: 0.4831 - val_acc: 0.8690
Epoch 64/125
390/390 [==============================] - 37s 94ms/step - loss: 0.2170 - acc: 0.9238 - val_loss: 0.4440 - val_acc: 0.8769
Epoch 65/125
390/390 [==============================] - 37s 94ms/step - loss: 0.2186 - acc: 0.9219 - val_loss: 0.4198 - val_acc: 0.8844
Epoch 66/125
390/390 [==============================] - 37s 94ms/step - loss: 0.2203 - acc: 0.9231 - val_loss: 0.4683 - val_acc: 0.8764
Epoch 67/125
390/390 [==============================] - 37s 94ms/step - loss: 0.2174 - acc: 0.9245 - val_loss: 0.4119 - val_acc: 0.8848
Epoch 68/125
390/390 [==========================

390/390 [==============================] - 37s 94ms/step - loss: 0.1740 - acc: 0.9376 - val_loss: 0.4409 - val_acc: 0.8842
Epoch 122/125
390/390 [==============================] - 36s 93ms/step - loss: 0.1757 - acc: 0.9377 - val_loss: 0.4519 - val_acc: 0.8821
Epoch 123/125
390/390 [==============================] - 37s 94ms/step - loss: 0.1767 - acc: 0.9384 - val_loss: 0.4370 - val_acc: 0.8857
Epoch 124/125
390/390 [==============================] - 37s 94ms/step - loss: 0.1752 - acc: 0.9383 - val_loss: 0.4460 - val_acc: 0.8835
Epoch 125/125
390/390 [==============================] - 36s 93ms/step - loss: 0.1786 - acc: 0.9364 - val_loss: 0.4421 - val_acc: 0.8846


{'acc': [0.46130093039133857,
  0.62798764836072996,
  0.68954523576541249,
  0.72413378252140181,
  0.74378408730163914,
  0.76197064481257326,
  0.77931504649996641,
  0.79415303172306406,
  0.80219361561783464,
  0.80504090471607315,
  0.81496631380147877,
  0.82136268850163763,
  0.82713747190901354,
  0.83180943214629455,
  0.83439605393622374,
  0.83790503687532736,
  0.83928857878075225,
  0.84446182228412081,
  0.84961501445608101,
  0.8535851780558229,
  0.85462784726993757,
  0.85675328838010611,
  0.85921960218158488,
  0.86054299003541723,
  0.86515479627847292,
  0.86643808145036594,
  0.86806223929444681,
  0.8701676291496967,
  0.87267404555662498,
  0.87427815203105252,
  0.87461902474148523,
  0.87810795636830286,
  0.87768429487179489,
  0.88003773276210218,
  0.87758662173256186,
  0.88175729870375508,
  0.88141642608893467,
  0.88685033688149006,
  0.88528633301251203,
  0.88574751359666049,
  0.8887953159898605,
  0.88853464873891863,
  0.88831408407430368,
  0.891

In [14]:
def run(act):# SWISH - 125
    init_shape = (3, 32, 32) if K.image_dim_ordering() == 'th' else (32, 32, 3)
    # For WRN-16-8 put N = 2, k = 8
    # For WRN-28-10 put N = 4, k = 10
    # For WRN-40-4 put N = 6, k = 4
    model = wrn.build_model(16, 4, init_shape, num_classes)
    model.summary()

    print("Model Created")
    batch_size  = 128
    epochs = 150

    opt = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.0, nesterov=False)
    lr_1 = keras.callbacks.LearningRateScheduler(schedule)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    print("Finished compiling")

    ####################
    # Network training #
    ####################

    print("Gonna fit the model")
#     his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
#     print(his.history)
    return his.history

In [13]:
run("act")

Model Created
Finished compiling
Gonna fit the model
Epoch 1/150
390/390 [==============================] - 84s 215ms/step - loss: 1.5876 - acc: 0.4164 - val_loss: 1.7169 - val_acc: 0.4275
Epoch 2/150
390/390 [==============================] - 84s 214ms/step - loss: 1.1048 - acc: 0.6065 - val_loss: 1.3743 - val_acc: 0.5580
Epoch 3/150
390/390 [==============================] - 84s 215ms/step - loss: 0.8679 - acc: 0.6948 - val_loss: 0.9323 - val_acc: 0.6794
Epoch 4/150
390/390 [==============================] - 84s 215ms/step - loss: 0.7192 - acc: 0.7482 - val_loss: 0.8433 - val_acc: 0.7124
Epoch 5/150
390/390 [==============================] - 84s 215ms/step - loss: 0.6179 - acc: 0.7842 - val_loss: 0.6428 - val_acc: 0.7787
Epoch 6/150
390/390 [==============================] - 84s 215ms/step - loss: 0.5489 - acc: 0.8103 - val_loss: 0.5743 - val_acc: 0.8018
Epoch 7/150
390/390 [==============================] - 84s 215ms/step - loss: 0.4939 - acc: 0.8293 - val_loss: 0.5800 - val_acc: 0.

390/390 [==============================] - 83s 214ms/step - loss: 0.0154 - acc: 0.9951 - val_loss: 0.2985 - val_acc: 0.9300
Epoch 61/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0136 - acc: 0.9959 - val_loss: 0.2961 - val_acc: 0.9321
Epoch 62/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0134 - acc: 0.9960 - val_loss: 0.3121 - val_acc: 0.9289
Epoch 63/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0149 - acc: 0.9954 - val_loss: 0.2988 - val_acc: 0.9310
Epoch 64/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0141 - acc: 0.9959 - val_loss: 0.3201 - val_acc: 0.9298
Epoch 65/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0146 - acc: 0.9953 - val_loss: 0.3149 - val_acc: 0.9300
Epoch 66/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0129 - acc: 0.9960 - val_loss: 0.3051 - val_acc: 0.9320
Epoch 67/150
390/390 [==============================] 

390/390 [==============================] - 83s 214ms/step - loss: 0.0057 - acc: 0.9984 - val_loss: 0.3272 - val_acc: 0.9318
Epoch 120/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0064 - acc: 0.9981 - val_loss: 0.3263 - val_acc: 0.9320
Epoch 121/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0064 - acc: 0.9982 - val_loss: 0.3275 - val_acc: 0.9319
Epoch 122/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0059 - acc: 0.9983 - val_loss: 0.3276 - val_acc: 0.9322
Epoch 123/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0054 - acc: 0.9984 - val_loss: 0.3266 - val_acc: 0.9323
Epoch 124/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0064 - acc: 0.9981 - val_loss: 0.3269 - val_acc: 0.9320
Epoch 125/150
390/390 [==============================] - 83s 214ms/step - loss: 0.0068 - acc: 0.9978 - val_loss: 0.3278 - val_acc: 0.9328
Epoch 126/150
390/390 [=========================

{'acc': [0.41630574268219295,
  0.60651267244145013,
  0.69475858200808771,
  0.74823548285518271,
  0.78426772535784262,
  0.81027430217542207,
  0.82930301572024379,
  0.84213586784074579,
  0.8568334937822295,
  0.86469361561783464,
  0.87662415782495839,
  0.88438402311825626,
  0.89066008979171984,
  0.89575312800769968,
  0.90295155594507237,
  0.9095484439975603,
  0.91167388512685121,
  0.92001523899274795,
  0.92314324673070414,
  0.92705325633622071,
  0.93144449791466155,
  0.93397096567212068,
  0.93511389154327729,
  0.94102903428963447,
  0.94126965028565779,
  0.94509945456554079,
  0.94610202115508357,
  0.94858838626884823,
  0.95128205128205123,
  0.95399807323686725,
  0.95662897012537396,
  0.957451074732241,
  0.95885466798819674,
  0.96067933906987191,
  0.96242380497889291,
  0.96466955407751187,
  0.96410811675983166,
  0.9655718639335229,
  0.96697545720860101,
  0.97006336222637302,
  0.97178777668925098,
  0.97028392683362064,
  0.97108598007083435,
  0.97397